# Notebook 2 - SQL avec vraies bases de données
## Analyse e-commerce avec PostgreSQL en ligne




### 🎯 Objectifs pédagogiques
- Connecter Python à une vraie base de données PostgreSQL
- Écrire des requêtes SQL complexes sur des données réelles
- Implémenter des analyses RFM avec SQL
- Intégrer SQL et pandas pour des analyses avancées
- Gérer les connexions et la sécurité

### 🛍️ Contexte du projet
Vous analysez les données d'un vrai dataset e-commerce (Brazilian E-Commerce Public Dataset) hébergé sur une base PostgreSQL.

Objectif : créer une segmentation clientèle pour optimiser les campagnes marketing.


## Partie 1 : Connexion à la base de données réelle

### 🔧 Installation et configuration


# Installation des dépendances


```
pip install psycopg2-binary sqlalchemy pandas python-dotenv
```




In [1]:
import psycopg2
import pandas as pd
import numpy as np
import requests
import zipfile
import io
import plotly.express as px
from sqlalchemy import create_engine, text
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import os
from dotenv import load_dotenv

### 🌐 Base de données PostgreSQL gratuite (ElephantSQL)

**Option 1 : ElephantSQL (20MB gratuit)**
1. Créez un compte sur [elephantsql.com](https://www.elephantsql.com/)
2. Créez une instance "Tiny Turtle" (gratuite)
3. Récupérez vos credentials

**Option 2 : Supabase (500MB gratuit)**
1. Créez un compte sur [supabase.com](https://supabase.com/)
2. Créez un nouveau projet
3. Récupérez l'URL de connexion PostgreSQL

In [2]:
# Configuration de connexion (à adapter selon votre provider)
load_dotenv()

USER = os.getenv("user")
PWD = os.getenv("password")
HOST = os.getenv("host")
PORT = os.getenv("port")
DBNAME = os.getenv("dbname")

# Création de l'URL SQAlchemy
DATABASE_URL = f"postgresql://{USER}:{PWD}@{HOST}:{PORT}/{DBNAME}"

engine = create_engine(DATABASE_URL, future=True, echo=False)

try:
    with engine.connect() as connection:
        print("Connection successful!")
except Exception as e:
    print(f"Failed to connect: {e}")

conn = engine.connect()

Connection successful!



## Partie 2 : Import du dataset e-commerce

### 📊 Dataset Brazilian E-Commerce
Nous utilisons le célèbre dataset Olist (100k commandes réelles).

**Tables à créer :**
1. **customers** : customer_id, customer_city, customer_state
2. **orders** : order_id, customer_id, order_status, order_date, order_delivered_date
3. **order_items** : order_id, product_id, seller_id, price, freight_value
4. **products** : product_id, product_category, product_weight_g
5. **sellers** : seller_id, seller_city, seller_state

In [3]:
### 🗃️ Création des tables SQL

# Load les CSV et créer les DataFrame pour chaque CSV

list_df =[]
for file in sorted(os.listdir("dataset_olist")):
    list_df.append(pd.read_csv(f"dataset_olist/{file}"))
df_cust = list_df[0]
df_orders_it = list_df[2]
df_ord = list_df[5]
df_prod = list_df[6]
df_sellers = list_df[7]

In [4]:
# On peut drop les colonnes non utiles
df_cust = df_cust[["customer_id", "customer_city", "customer_state"]]
df_ord = df_ord[["order_id", "customer_id", "order_status", "order_purchase_timestamp","order_delivered_customer_date"]]
df_ord = df_ord.rename(columns={"order_purchase_timestamp":"order_date", "order_delivered_customer_date": "order_delivered_date"})
df_orders_it = df_orders_it[["order_id", "product_id", "seller_id", "price","freight_value"]]
df_prod = df_prod[["product_id", "product_category_name", "product_weight_g"]]
df_prod = df_prod.rename(columns={"product_category_name":"product_category"})
df_sellers = df_sellers[["seller_id", "seller_city", "seller_state"]]

In [5]:
# Créer une fonction pour exporter les DataFrame directement avec la commande pandas to_sql

def tables_to_sql():
    df_cust.to_sql(name="customers",con=engine,index=False,if_exists='append') 
    df_ord.to_sql(name="orders",con=engine,index=False,if_exists='append')
    df_prod.to_sql(name="products",con=engine,index=False,if_exists='append') 
    df_sellers.to_sql(name="sellers",con=engine,index=False, if_exists='append')
    df_orders_it.to_sql(name="order_items",con=engine,index=False,if_exists='append')

In [6]:

# Créez les tables dans PostgreSQL

# Tips :
# - Utilisez des SERIAL pour les IDs auto-increment
# - Ajoutez des index sur les clés étrangères
# - Incluez des contraintes de validation


create_customers = """
CREATE TABLE IF NOT EXISTS customers (
customer_id VARCHAR(50) PRIMARY KEY,
customer_city VARCHAR(100),
customer_state VARCHAR(2)
);
"""
create_orders = """
CREATE TABLE IF NOT EXISTS orders (
order_id VARCHAR(50) PRIMARY KEY,
customer_id VARCHAR(50),
order_status VARCHAR(20) CHECK (order_status IN (
    'created', 'shipped', 'delivered', 'canceled', 'invoiced', 'processing', 'unavailable', 'approved'
)),
order_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
order_delivered_date TIMESTAMP,
CONSTRAINT fk_customer
    FOREIGN KEY(customer_id)
        REFERENCES customers(customer_id)
);
"""
create_products = """ 
CREATE TABLE IF NOT EXISTS products (
product_id VARCHAR(50) PRIMARY KEY,
product_category VARCHAR(50),
product_weight_g NUMERIC CHECK (product_weight_g >=0)  
);
"""
create_order_items = """ 
CREATE TABLE IF NOT EXISTS order_items (
order_id VARCHAR(50),
product_id VARCHAR(50),
seller_id VARCHAR(50),
price NUMERIC(10,2) CHECK (price >= 0),
freight_value NUMERIC(10,2) CHECK (freight_value >= 0),
CONSTRAINT fk_product
    FOREIGN KEY(product_id)
        REFERENCES products(product_id),
CONSTRAINT fk_seller
    FOREIGN KEY(seller_id)
        REFERENCES sellers(seller_id),
CONSTRAINT fk_order
    FOREIGN KEY(order_id)
        REFERENCES orders(order_id)
);
"""
create_sellers = """
CREATE TABLE IF NOT EXISTS sellers (
seller_id VARCHAR(50) PRIMARY KEY,
seller_city VARCHAR(50),
seller_state VARCHAR(2)
);
"""
# Complétez pour les autres tables
# N'oubliez pas les contraintes de clés étrangères !


# with engine.connect() as con:
#     con.execute(sa.text(create_customers))
#     con.execute(sa.text(create_orders))
#     con.execute(sa.text(create_products))
#     con.execute(sa.text(create_sellers))
#     con.execute(sa.text(create_order_items))
# con.commit()

conn.execute(text(create_customers))
conn.execute(text(create_orders))
conn.execute(text(create_products))
conn.execute(text(create_sellers))
conn.execute(text(create_order_items))
conn.commit()

In [7]:
%%script echo skipping
# Clear le schema (et toutes les tables) si besoin
clear_all = """
DROP SCHEMA public CASCADE;
CREATE SCHEMA public;
"""
conn.execute(text(clear_all))
conn.commit()

skipping


In [8]:
%%script echo skipping
# On insert les données grâce à notre fonction

tables_to_sql()

skipping


## Partie 3 : Requêtes SQL avancées


### 🔍 Analyses SQL à implémenter

#### 1. Analyse RFM (Récence, Fréquence, Montant)
```sql
-- Votre défi : Calculer les métriques RFM pour chaque client
WITH customer_metrics AS (
    SELECT
        c.customer_id,
        c.customer_state,
        -- Récence : jours depuis dernier achat
        -- Fréquence : nombre de commandes
        -- Montant : total dépensé
        
        -- Complétez cette requête CTE
        
    FROM customers c
    JOIN orders o ON c.customer_id = o.customer_id
    JOIN order_items oi ON o.order_id = oi.order_id
    WHERE o.order_status = 'delivered'
    GROUP BY c.customer_id, c.customer_state
)

-- Créez les segments RFM (Champions, Loyaux, À risque, etc.)
SELECT
    customer_id,
    customer_state,
    recency_score,
    frequency_score,
    monetary_score,
    CASE
        WHEN recency_score >= 4 AND frequency_score >= 4 THEN 'Champions'
        WHEN recency_score >= 3 AND frequency_score >= 3 THEN 'Loyal Customers'
        -- Ajoutez les autres segments
        ELSE 'Others'
    END as customer_segment
FROM customer_metrics;
```

In [9]:
#### 1. Analyse RFM
# Votre défi : Calculer les métriques RFM pour chaque client

### Récence : jours depuis dernier achat du client
# Fréquence : nombre de commandes du client
# Montant : total dépensé du client
        
# Complétez cette requête CTE

conn.rollback()

customer_metrics = """
WITH base_data AS (
    SELECT
        c.customer_id,
        c.customer_state,
        MAX(o.order_date) AS last_order_date,
        COUNT(o.order_id) AS order_count,
        SUM(oi.price) AS total_spent
    FROM
        customers c
    JOIN orders o ON c.customer_id = o.customer_id
    JOIN order_items oi ON o.order_id = oi.order_id
    WHERE
        o.order_status = 'delivered'
    GROUP BY
        c.customer_id, c.customer_state
),
scored_data AS (
    SELECT
        customer_id,
        customer_state,
        last_order_date,
        order_count,
        total_spent,
        NTILE(5) OVER (ORDER BY last_order_date DESC) AS recency_score,
        NTILE(5) OVER (ORDER BY order_count) AS frequency_score,
        NTILE(5) OVER (ORDER BY total_spent) AS monetary_score
    FROM
        base_data
)
SELECT
    customer_id,
    customer_state,
    recency_score,
    frequency_score,
    monetary_score,
    CASE
        WHEN recency_score >= 4 AND frequency_score >= 4 AND monetary_score >= 4 THEN 'Champions'
        WHEN recency_score >= 3 AND frequency_score >= 3 THEN 'Loyal Customers'
        WHEN recency_score >= 4 AND frequency_score BETWEEN 2 AND 3 THEN 'Potential Loyalists'
        WHEN recency_score BETWEEN 2 AND 3 AND frequency_score >= 3 THEN 'At risk'
        WHEN recency_score <= 2 AND frequency_score <= 3 THEN 'Hibernating'
        WHEN recency_score = 1 THEN 'Lost'
        ELSE 'Others'
    END AS customer_segment
FROM
    scored_data;
"""

result_cm= conn.execute(text(customer_metrics))
df_cm = pd.DataFrame(result_cm.fetchall())
df_cm

,customer_id,customer_state,recency_score,frequency_score,monetary_score,customer_segment
0,898b7fee99c4e42170ab69ba59be0a8b,SP,1,4,3,Lost
1,496630b6740bcca28fce9ba50d8a26ef,SP,1,1,1,Hibernating
2,6e353700bc7bcdf6ebc15d6de16d7002,MG,1,2,2,Hibernating
3,e60df9449653a95af4549bbfcb18a6eb,PR,1,5,5,Lost
4,e450a297a7bc6839ceb0cf1a2377fa02,SP,1,2,2,Hibernating
...,...,...,...,...,...,...
96473,b8cf418e97ae795672d326288dfab7a7,SP,5,3,4,Loyal Customers
96474,6f989332712d3222b6571b1cf5b835ce,RS,5,1,1,Others
96475,7ec40b22510fdbea1b08921dd39e63d8,RS,5,1,1,Others
96476,355077684019f7f60a031656bd7262b8,SP,5,1,1,Others


In [10]:
#### 2. Analyse géographique des ventes


# Analysez les performances par état/région

# Requêtes à écrire :
# 1. Top 10 des états par CA
# 2. Croissance MoM du CA par région
# 3. Taux de conversion des par ville
# 4. Distance moyenne vendeur-acheteur

conn.rollback()

query_top_states = """
WITH base_data AS(
    SELECT
        c.customer_id,
        c.customer_state,
        SUM(oi.price) AS total_spent
    FROM
        customers c
    JOIN orders o ON c.customer_id = o.customer_id
    JOIN order_items oi ON o.order_id = oi.order_id
    WHERE
        o.order_status = 'delivered'
    GROUP BY
        c.customer_id, c.customer_state
)
SELECT
    customer_state,
    SUM(total_spent) AS CA_total_spent
FROM base_data
GROUP BY customer_state
ORDER BY CA_total_spent DESC
LIMIT 10;
    
-- Calculez le CA, nombre de commandes, panier moyen
"""

pd.read_sql(query_top_states, engine)

,customer_state,ca_total_spent
0,SP,5067633.16
1,RJ,1759651.13
2,MG,1552481.83
3,RS,728897.47
4,PR,666063.51
5,SC,507012.13
6,BA,493584.14
7,DF,296498.41
8,GO,282836.70
9,ES,268643.45


In [11]:
conn.rollback()

query_MoM = """
WITH base_data AS (
    SELECT
        c.customer_id,
        c.customer_state,
        SUM(oi.price) AS total_spent,
        DATE_TRUNC('month', o.order_date) AS month
    FROM
        customers c
    JOIN orders o ON c.customer_id = o.customer_id
    JOIN order_items oi ON o.order_id = oi.order_id
    WHERE
        o.order_status = 'delivered'
    GROUP BY
        c.customer_id, c.customer_state, DATE_TRUNC('month', o.order_date)
), 
MoM_data AS (
    SELECT
        customer_state,
        month,
        SUM(total_spent) AS CA
    FROM base_data
    GROUP BY customer_state, month
)
SELECT
    customer_state,
    month,
    CA AS month_CA,
    LAG(CA) OVER (PARTITION BY customer_state ORDER BY month) AS previous_month_CA,
    ROUND(
        CASE 
            WHEN LAG(CA) OVER (PARTITION BY customer_state ORDER BY month) = 0 THEN NULL
            ELSE 100.0 * (CA - LAG(CA) OVER (PARTITION BY customer_state ORDER BY month)) / 
                         LAG(CA) OVER (PARTITION BY customer_state ORDER BY month)
        END, 2
    ) AS mom_growth_percent
FROM MoM_data
ORDER BY customer_state, month DESC;

    
    
-- Calculez le CA, nombre de commandes, panier moyen
"""
pd.read_sql(query_MoM, engine)

,customer_state,month,month_ca,previous_month_ca,mom_growth_percent
0,AC,2018-08-01,446.69,1469.97,-69.61
1,AC,2018-07-01,1469.97,427.90,243.53
2,AC,2018-06-01,427.90,1364.89,-68.65
3,AC,2018-05-01,1364.89,422.98,222.68
4,AC,2018-04-01,422.98,97.80,332.49
...,...,...,...,...,...
551,TO,2017-05-01,2073.43,2843.26,-27.08
552,TO,2017-04-01,2843.26,648.66,338.33
553,TO,2017-03-01,648.66,765.60,-15.27
554,TO,2017-02-01,765.60,629.89,21.55


In [12]:
conn.rollback()

query_taux_conversion = """
WITH base_data AS (
    SELECT
        c.customer_id,
        c.customer_city,
        SUM(CASE WHEN o.order_status = 'delivered' THEN 1 ELSE 0 END) AS successful,
        SUM(CASE WHEN o.order_status IN (
            'created', 'shipped', 'canceled', 'invoiced',
            'processing', 'unavailable', 'approved'
        ) THEN 1 ELSE 0 END) AS unsuccessful
    FROM
        customers c
    JOIN orders o ON c.customer_id = o.customer_id
    JOIN order_items oi ON o.order_id = oi.order_id
    GROUP BY
        c.customer_id, c.customer_city
)
SELECT
    customer_city,
    ROUND(
        CASE 
            WHEN SUM(unsuccessful) = 0 THEN 100
            ELSE 100.0 * SUM(successful) / (SUM(successful) + SUM(unsuccessful))
        END, 2
    ) AS conversion_rate_percent
FROM base_data
GROUP BY
    customer_city
ORDER BY 
    conversion_rate_percent DESC;

-- Calculez le CA, nombre de commandes, panier moyen
"""
pd.read_sql(query_taux_conversion, engine)

,customer_city,conversion_rate_percent
0,balsa nova,100.0
1,alto rio doce,100.0
2,alvorada do gurgueia,100.0
3,monte alegre do sul,100.0
4,maracai,100.0
...,...,...
4105,cacimbinhas,0.0
4106,nova brescia,0.0
4107,lebon regis,0.0
4108,santo antonio de goias,0.0


#### 3. Analyse temporelle et saisonnalité
```sql
-- Détectez les patterns saisonniers
SELECT
    EXTRACT(YEAR FROM order_date) as year,
    EXTRACT(MONTH FROM order_date) as month,
    EXTRACT(DOW FROM order_date) as day_of_week,
    COUNT(*) as order_count,
    SUM(price + freight_value) as total_revenue,
    AVG(price + freight_value) as avg_order_value
FROM orders o
JOIN order_items oi ON o.order_id = oi.order_id
WHERE order_status = 'delivered'
GROUP BY ROLLUP(
    EXTRACT(YEAR FROM order_date),
    EXTRACT(MONTH FROM order_date),
    EXTRACT(DOW FROM order_date)
)
ORDER BY year, month, day_of_week;
```

---

In [13]:
query_seasonal_patterns = """
SELECT
    EXTRACT(YEAR FROM order_date) as year,
    EXTRACT(MONTH FROM order_date) as month,
    EXTRACT(DOW FROM order_date) as day_of_week,
    COUNT(*) as order_count,
    SUM(price + freight_value) as total_revenue,
    AVG(price + freight_value) as avg_order_value
FROM orders o
JOIN order_items oi ON o.order_id = oi.order_id
WHERE order_status = 'delivered'
GROUP BY ROLLUP(
    EXTRACT(YEAR FROM order_date),
    EXTRACT(MONTH FROM order_date),
    EXTRACT(DOW FROM order_date)
)
ORDER BY year, month, day_of_week;
"""
data_seasonal = pd.read_sql_query(query_seasonal_patterns, engine)
data_seasonal

,year,month,day_of_week,order_count,total_revenue,avg_order_value
0,2016.0,9.0,4.0,3,143.46,47.820000
1,2016.0,9.0,NaN,3,143.46,47.820000
2,2016.0,10.0,0.0,26,2904.19,111.699615
3,2016.0,10.0,1.0,44,4456.26,101.278636
4,2016.0,10.0,2.0,63,9821.42,155.895556
...,...,...,...,...,...,...
171,2018.0,8.0,5.0,1003,135803.41,135.397218
172,2018.0,8.0,6.0,758,110963.05,146.389248
173,2018.0,8.0,NaN,7142,985491.64,137.985388
174,2018.0,NaN,NaN,60324,8451584.77,140.103189


In [ ]:
fig = px.line(
    data_seasonal,
    x='month',
    y='order_count',
    title='Évolution du nombre de commandes (saisonnalité mensuelle)',
    labels={'year_month': 'Date', 'order_count': 'Nombre de commandes'}
)

fig.show()

In [20]:
fig = px.line(
    data_seasonal,
    x='month',
    y='total_revenue',
    title='Évolution du chiffre d’affaires (saisonnalité mensuelle)',
    labels={'year_month': 'Date', 'total_revenue': 'Revenu total'}
)

fig.show()

In [21]:
fig = px.box(
    data_seasonal,
    x='day_of_week',
    y='avg_order_value',
    points='all',
    title='Valeur moyenne des commandes par jour de la semaine',
    labels={'day_of_week': 'Jour de la semaine (0 = Dimanche)', 'avg_order_value': 'Valeur moyenne'}
)
fig.show()

## Partie 4 : Analyse prédictive avec SQL

### 🔮 Modèles simples en SQL

In [15]:
#### 1. Prédiction de churn

def churn_prediction_sql():
    """
    Identifiez les clients à risque de churn

    Indicateurs :
    - Pas d'achat depuis X jours
    - Baisse de fréquence d'achat
    - Diminution du panier moyen
    - Changement de comportement géographique
    """

    churn_query = """
    WITH customer_activity AS (
        -- Calculez les métriques d'activité récente
        -- Comparez avec l'historique du client
        -- Scorez le risque de churn
    )

    SELECT
        customer_id,
        days_since_last_order,
        order_frequency_trend,
        monetary_trend,
        churn_risk_score,
        CASE
            WHEN churn_risk_score > 0.7 THEN 'High Risk'
            WHEN churn_risk_score > 0.4 THEN 'Medium Risk'
            ELSE 'Low Risk'
        END as churn_segment
    FROM customer_activity;
    """

    return pd.read_sql(churn_query, engine)


#### 2. Recommandations produits
```sql
-- Market Basket Analysis simplifié
WITH product_pairs AS (
    SELECT
        oi1.product_id as product_a,
        oi2.product_id as product_b,
        COUNT(*) as co_purchase_count
    FROM order_items oi1
    JOIN order_items oi2 ON oi1.order_id = oi2.order_id
    WHERE oi1.product_id != oi2.product_id
    GROUP BY oi1.product_id, oi2.product_id
    HAVING COUNT(*) >= 10  -- Seuil minimum
)

SELECT
    product_a,
    product_b,
    co_purchase_count,
    co_purchase_count::float / total_a.count as confidence
FROM product_pairs pp
JOIN (
    SELECT product_id, COUNT(*) as count
    FROM order_items
    GROUP BY product_id
) total_a ON pp.product_a = total_a.product_id
ORDER BY confidence DESC;
```

---

## Partie 5 : Intégration avec les APIs météo

### 🌤️ Croisement données météo/ventes
```python
def weather_sales_correlation():
    """
    Correlez vos données météo du Notebook 1 avec les ventes
    
    Hypothèses à tester :
    1. Les ventes de certaines catégories augmentent-elles avec la pluie ?
    2. Y a-t-il un impact de la température sur les achats ?
    3. Les livraisons sont-elles impactées par la météo ?
    """
    
    # Récupérez les données météo historiques pour les villes brésiliennes
    weather_query = """
    SELECT DISTINCT customer_city, customer_state
    FROM customers
    WHERE customer_state IN ('SP', 'RJ', 'MG', 'RS', 'SC')
    ORDER BY customer_city;
    """
    
    cities = pd.read_sql(weather_query, engine)
    
    # Intégrez avec l'API météo
    # Analysez les corrélations
    
    pass
```

### 📊 Dashboard géo-temporel
```python
def create_geotemporal_dashboard():
    """
    Créez un dashboard interactif combinant :
    - Carte des ventes par région
    - Évolution temporelle avec météo
    - Segments clients géolocalisés
    - Prédictions par zone géographique
    """
    pass
```

---
## 🏆 Livrables finaux

### 📈 Rapport d'analyse complet
1. **Segmentation RFM (Recency, Frenquency, Monetary) ** : 5-7 segments avec caractéristiques
2. **Analyse géographique**  : Performances par région + recommandations
3. **Prédictions churn** : Liste des clients à risque + actions
4. **Recommandations produits** : Top 10 des associations
5. **Impact météo** : Corrélations significatives identifiées

### 🚀 Pipeline automatisé
```python
def automated_analysis_pipeline():
    """
    Pipeline qui :
    1. Se connecte à la DB
    2. Exécute toutes les analyses
    3. Met à jour les segments clients
    4. Génère le rapport automatiquement
    5. Envoie des alertes si nécessaire
    """
    pass
```

---

## 🎓 Auto-évaluation

- [ ] **Connexion DB** : PostgreSQL fonctionnelle
- [ ] **Requêtes complexes** : JOINs, CTEs, fonctions analytiques
- [ ] **Gestion des erreurs** : Connexions robustes
- [ ] **Performance** : Requêtes optimisées avec index
- [ ] **Intégration** : SQL + Python + APIs
- [ ] **Insights actionables** : Recommandations business claires

### 🔗 Préparation au Notebook 3
Le prochain notebook portera sur NoSQL (MongoDB) avec des données de réseaux sociaux et d'IoT, en temps réel.

### 💡 Bases de données alternatives
- **PlanetScale** : MySQL serverless gratuit
- **MongoDB Atlas** : 512MB gratuit
- **FaunaDB** : Base multi-modèle gratuite
- **Hasura Cloud** : GraphQL + PostgreSQL